In [1]:
from sample_entry import samples_sheet, metadata_sheet, log_sheet, work_list, check_list, cell, plans, plan_factory, scan
from IPython.display import display
from startup import det, motor
from bluesky.callbacks import LivePlot

In [2]:
%run startup.py
import ipywidgets as widgets
output = widgets.Output()

In [3]:
print('SAMPLES DATABASE')
display(samples_sheet)
print('\nPLANS LIST')
display(metadata_sheet)
print('\nHISTORY')
display(widgets.VBox([log_sheet, output]))

SAMPLES DATABASE


Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='a'), Cell(column_e…


PLANS LIST


Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, style={'backgroundColor': 'grey'}, typ…


HISTORY


In [5]:
s = RE(plan_factory(work_list))

Lock row 1 and 🚀
Lock row 2 and 🚀
Skip row 3
Skip row 4
Skip row 5
Skip row 6


In [11]:
s

('31be8750-6d95-415e-924d-7d4fb641d6ca',)

In [9]:
%matplotlib widget
import matplotlib.pyplot as plt

f, axarr = plt.subplots(1,3)

axarr[0,0].imshow(db[0].primary.read().det_img.plot())
axarr[0,1].imshow(db[1].primary.read().det_img.plot())
axarr[0,2].imshow(db[2].primary.read().det_img.plot())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: too many indices for array